In [1]:
%load_ext autoreload
%autoreload 2

from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,balanced_accuracy_score,f1_score
from answer_judge.vqaEval import VQAEval
import json

/root/miniconda3/envs/VL-Uncertainty/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('/home/hallscope/VL_Uncertainty/exp/log_2025_01_23_17_23_28_gen.json', 'r') as file: 
    data = json.load(file)

In [3]:
df=pd.DataFrame(data)
df = df.drop(columns=['args','begin_time_str','Total samples','end_time_str'])

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
question,"As of 2021, how many championship titles had F...",What game topped the charts with 512.3 million...,How many hours did League of Legends watch on ...,Which country was the leading pharmaceutical s...,What is the region with the highest number of ...,What was the global sales income of certified ...,What percentage of Belize's GDP did the nation...,In what year did the Mexican government spend ...,How much did the Mexican government spend in t...,What percentage of the GDP of the United State...,...,What was Cineplex's revenue a year earlier?\nA...,How many males live in Germany?\nAnswer the qu...,How many females live in Germany?\nAnswer the ...,How many posts per day did sports teams post o...,How many touchdowns has Mike Alstott scored fo...,Who is the career touchdown leader of the Tamp...,Who has played the most games in Toronto Blue ...,How many seats did Mark Rutte's VVD have?\nAns...,What was the average revenue per available roo...,How many children spoke another language at ho...
gt_ans,16,League of Legends,512.3,Switzerland,Lombardy,76.4,97.52,2020,6.61,9.23,...,1665.15,41037613,42129098,1.76,71,Mike Alstott,Tony Fernandez,33,42.36,12.13
flag_sample_valid,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
ans,16,League of Legends,512.3,Switzerland,Lombardy,76.4,97.52,2020,6.65,9.23,...,1665.15,41037613,42129098,1.76,71,Mike Alstott,Tony Fernandez,33,42.36,12.23
perturbed_question_list,[How many championship titles did Ferrari win ...,[Which game was the most popular on Twitch in ...,"[""How much time was spent watching League of L...",[What was the top pharmaceutical supplier to G...,[Which is the area in Italy that has the most ...,[What was the worldwide revenue from Fairtrade...,"[In 2019, what proportion of Belize's GDP was ...",[What was the year when the Mexican government...,[What was the amount of money that the Mexican...,"[In 1990, what proportion of the GDP of the Un...",...,[What was Cineplex's revenue in the previous f...,[What is the male population of Germany?\nAnsw...,[What is the population of females living in G...,[What is the average number of posts per day t...,[What is the total number of touchdowns that M...,[Who holds the record for the most career touc...,[Which player holds the record for the most ga...,[What was the number of seats that Mark Rutte'...,[What was the average revenue per available ro...,"[In 2018, how many children in the United Stat..."


In [5]:
def get_ans(x):
    return [y['answer'] for y in x]

gt_label=df.loc['gt_ans'].to_list()
#gt_label

In [6]:
rsps=df.loc['ans'].to_list()
#rsps

In [7]:
from answer_judge import load_judge

In [8]:
gt_label[0]

'16'

In [9]:
is_correct=[]
judge=load_judge('ChartQA')
for i,x in enumerate(rsps):
    if judge.check(x,gt_label[i]):
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [10]:
is_correct.shape

(1000,)

In [11]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 1000 responses. 790 answer correctly and 210 answer wrongly


In [12]:
%load_ext autoreload
%autoreload 2
from VL_Uncertainty import uncertainty_estimation,hallucination_detection,obtain_lvlm,obtain_llm

class Args: 
    def __init__(self): 
        self.uncertainty_thres=1
        self.benchmark = 'open_end'
        self.llm='Qwen2.5-3B'
        self.image_shape=None
        self.sampling_time=5
        
args=Args()

llm=obtain_llm(args)
uncertainty=[]
detection_flag=[]
detect_preds=[]

num_samples=2000
for i in tqdm(range(int(data['Total samples']))):
    idx=str(i)
    data[idx]['flag_ans_correct']=True if is_correct[i]==1 else False
    uncertainty_estimation(args,idx,llm,data)
    uncertainty.append(data[idx]['uncertainty'])
    hallucination_detection(args,idx,data)
    detection_flag.append(data[idx]['flag_detection_correct'])
    detect_preds.append(data[idx]['flag_predict_hallucination'])
    if i >=num_samples:
        break

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.22it/s]
Some parameters are on the meta device because they were offloaded to the cpu.
  0%|          | 0/1000 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
  0%|          | 1/1000 [00:06<1:56:01,  6.97s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `e

KeyboardInterrupt: 

In [1]:
filename='exp/ChartVQA_checked.json'
with open(filename, 'w') as file: 
    json.dump(data, file)

NameError: name 'json' is not defined

In [13]:
import numpy as np

In [14]:
detect_preds=1-np.array(detect_preds)

In [15]:
detect_preds.shape

(201,)

In [16]:
uncertainty=-np.array(uncertainty)

In [17]:
l=detect_preds.shape[0]

In [19]:
sum(is_correct[:l]==1)

162

In [18]:
print(f'acc: {accuracy_score(is_correct[:l],detect_preds)} f1:{f1_score(is_correct[:l],detect_preds)} auroc:{roc_auc_score(is_correct[:l],uncertainty)} balanced_accuracy_score:{balanced_accuracy_score(is_correct[:l],detect_preds)}')

acc: 0.8756218905472637 f1:0.9235474006116208 auroc:0.8311965811965811 balanced_accuracy_score:0.7865622032288699
